In [1]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

# Датасеты

In [2]:
# https://drive.google.com/file/d/1ZawNKVV3ANYkF38R9MnXMOJOXZUxFjkp/view?usp=sharing
# https://drive.google.com/file/d/1WjhaG9dxm0-8hewIyoicq1Q5FucNy2yM/view?usp=sharing

# !gdown --id -O history.csv.gz 1ZawNKVV3ANYkF38R9MnXMOJOXZUxFjkp  # history
!gdown --id -O popularity.csv.gz 1WjhaG9dxm0-8hewIyoicq1Q5FucNy2yM  # popularity

Downloading...
From: https://drive.google.com/uc?id=1WjhaG9dxm0-8hewIyoicq1Q5FucNy2yM
To: /content/popularity.csv.gz
100% 4.25M/4.25M [00:00<00:00, 66.4MB/s]


In [3]:
# https://drive.google.com/file/d/1Gd5ezm3UBoqCpTgiu4Dd1bu1RqCb4AEm/view?usp=sharing
!gdown --id -O history_.csv 1Gd5ezm3UBoqCpTgiu4Dd1bu1RqCb4AEm  # history - подвыборка

Downloading...
From: https://drive.google.com/uc?id=1Gd5ezm3UBoqCpTgiu4Dd1bu1RqCb4AEm
To: /content/history_.csv
100% 437M/437M [00:02<00:00, 160MB/s]


# Датасет с данными о популярных запросах

In [4]:
import pandas as pd

popularity = pd.read_csv("popularity.csv.gz", #escapechar="\\", 
                         encoding="utf-8", error_bad_lines=False,
                         compression='gzip')
popularity.head()

,query,query_popularity
0,ноутбук,10
1,куртка женская осенняя,10
2,ботинки женские,10
3,видеокарта,10
4,пальто женское осеннее,10


Мы видим, что датасет действительно содержит популярные запросы

In [5]:
search_query = "попи"
popularity.loc[popularity["query"].apply(lambda x: search_query in str(x).lower())][:4]

,query,query_popularity
13365,попит,10
15435,попит гигант,10
107498,Попиты,7
111457,попит картошка,7


Датасет истории запросов за последнюю неделю. Подробная информация и EDA в отдельном файле

In [6]:
# history = pd.read_csv("history.csv.gz", encoding="utf-8", 
#                          error_bad_lines=False, compression='gzip')

history = pd.read_csv("history_.csv", encoding="utf-8", 
                         error_bad_lines=False)

print(history.shape)
history.head()

(5025541, 6)


,wbuser_id,UQ,cnt,locale,weekday,time
0,37bc0ce12ffabce1b1882e66d461ed0e,тапочки женские домашние,1933,Ru,0,10:48:53
1,4636a6706e6736d818816d8657565aa2,чехол для бейджика,1513,Ru,0,10:48:53
2,708f4040baf99acfc9496563edff1b1a,GUESS,4,Ru,0,10:48:53
3,70311ec9008a31f743c164e6f1198c86,штора для ванной,0,Ru,0,10:48:53
4,70311ec9008a31f743c164e6f1198c86,фототфон,92272,Ru,0,10:48:53


In [ ]:
popularity.to_csv("pop.csv", index=False)
history[["UQ", "cnt"]].to_csv("his.csv", index=False)

# Имплементация алгоритма **fast-autocomplete** *

** Алгоритм автозаполнения в поисковой строке с использованием ориентированного графа слов (DWG) и расстояния редактирования Левенштейна. Результаты кэшируются через LFU (наименее часто используемые).*

In [8]:
!pip install fast-autocomplete[levenshtein]

     |████████████████████████████████| 50 kB 2.8 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149864 sha256=3b08ace3aea9504533dc1618fc679bdbb859e134c652354bd35ef6681a03dd9d
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [9]:
import pandas as pd

def get_words(path_to_csv, column_name):
    words = {}
    df = pd.read_csv(path_to_csv, encoding="utf-8", error_bad_lines=False,)
    for search in df[column_name].values:
        search = str(search).strip().lower()
        search = ' '.join(search.split()[:4])
        words.update({search:{}})

    # words = {str(w).strip().lower():{} for w in df[column_name].values}
    return words

words_p = get_words("pop.csv", "query")
words_h = get_words("his.csv", "UQ")

In [10]:
for idx, item in enumerate(words_p.items()):
    print(item)
    if idx == 10:
      break

('ноутбук', {})
('куртка женская осенняя', {})
('ботинки женские', {})
('видеокарта', {})
('пальто женское осеннее', {})
('пальто женское', {})
('джинсы женские', {})
('кроссовки мужские', {})
('кроссовки женские', {})
('куртка мужская', {})
('айфон 11', {})


Обучаем fast autocomplete на запросах из датасета с историей запросов

In [11]:
from fast_autocomplete import AutoComplete

autocomplete = AutoComplete(words=words_h, 
                            valid_chars_for_string='абвгдеёжзиклмнопрстуфхцчщшьыъэюя')

Проверим как работает выдача (пока без ранжирования)

In [12]:
%time autocomplete.search(word='айф', max_cost=3, size=20)

CPU times: user 6.91 ms, sys: 0 ns, total: 6.91 ms
Wall time: 6.64 ms


[['а.ф.'],
 ['а.ф.', 'айфо'],
 ['а.ф.', 'айфт'],
 ['а.ф.', 'wu tang айфон'],
 ['а.ф.', 'айфое'],
 ['а.ф.', 'айфоны'],
 ['а.ф.', 'айфо4'],
 ['а.ф.', 'афган'],
 ['а.ф.', 'афина'],
 ['а.ф.', 'афиней'],
 ['а.ф.', 'афиша'],
 ['а.ф.', 'афасий'],
 ['а.ф.', 'афаки'],
 ['а.ф.', 'афнан'],
 ['а.ф.', 'айфон1'],
 ['а.ф.', 'айфон 6 s'],
 ['а.ф.', 'айфон 8 case'],
 ['а.ф.', 'айфон 7 .'],
 ['а.ф.', 'айфон 2'],
 ['а.ф.', 'айфон xе']]

In [13]:
%time autocomplete.search(word='пальто осе', max_cost=3, size=20)

CPU times: user 1.37 ms, sys: 34 µs, total: 1.4 ms
Wall time: 1.4 ms


[['jack wolfskin / пальто'],
 ['jack wolfskin / пальто', 'adona sandrelli пальто осень'],
 ['jack wolfskin / пальто', 'пальто осеннее elektra styil'],
 ['jack wolfskin / пальто', 'пальто осень п'],
 ['jack wolfskin / пальто', 'пальто осень 21'],
 ['jack wolfskin / пальто', 'пальто осеннее 60'],
 ['jack wolfskin / пальто', 'пальто осеннее на'],
 ['jack wolfskin / пальто', 'пальто осень-зима'],
 ['jack wolfskin / пальто', 'пальто осень 2021'],
 ['jack wolfskin / пальто', 'пальто осень серое'],
 ['jack wolfskin / пальто', 'пальто осень весна'],
 ['jack wolfskin / пальто', 'пальто осеннее драп'],
 ['jack wolfskin / пальто', 'пальто осеннее 2021'],
 ['jack wolfskin / пальто', 'пальто осеннее твоё'],
 ['jack wolfskin / пальто', 'пальто осень теплое'],
 ['jack wolfskin / пальто', 'пальто осень жёлтое'],
 ['jack wolfskin / пальто', 'пальто осень шерсть'],
 ['jack wolfskin / пальто', 'пальто осень прямое'],
 ['jack wolfskin / пальто', 'пальто осень балонь'],
 ['jack wolfskin / пальто', 'пальто 

In [14]:
%time autocomplete.search(word='плате', max_cost=3, size=20)  # mis-spelling

CPU times: user 1.47 ms, sys: 50 µs, total: 1.52 ms
Wall time: 1.53 ms


[['плате colombetta'],
 ['плате colombetta', 'платен'],
 ['плате colombetta', 'платее'],
 ['плате colombetta', 'платенца'],
 ['плате colombetta', 'платенцы'],
 ['плате colombetta', 'платенса'],
 ['плате colombetta', 'платесия'],
 ['плате colombetta', 'платеньцы'],
 ['плате colombetta', 'платешкаф'],
 ['плате colombetta', 'плате бохо'],
 ['плате colombetta', 'modis плате 2021'],
 ['плате colombetta', 'плате в пол'],
 ['плате colombetta', 'плате белое'],
 ['плате colombetta', 'плате зимни'],
 ['плате colombetta', 'плате шифон'],
 ['плате colombetta', 'плате 1 год'],
 ['плате colombetta', 'плате свитер'],
 ['плате colombetta', 'плате теплое'],
 ['плате colombetta', 'плате футляр'],
 ['плате colombetta', 'плате паетки']]

Пример задания весов

In [15]:
# example of setting weights
autocomplete.get_count_of_word('кроссовки')
autocomplete.update_count_of_word(word='кроссовки', count=10)
autocomplete.get_count_of_word('кроссовки')

10

In [16]:
def get_searches(word, size=100):
    res = autocomplete.search(word=word, max_cost=3, size=size)
    searches = []
    for search in res:
        for subsearch in search:
            if subsearch not in searches:
                searches.append(subsearch)
    return searches

Результат выдачи с использованием FA:

In [17]:
get_searches("пальто осе")

['jack wolfskin / пальто',
 'adona sandrelli пальто осень',
 'пальто осеннее elektra styil',
 'пальто осень п',
 'пальто осень 21',
 'пальто осеннее 60',
 'пальто осеннее на',
 'пальто осень-зима',
 'пальто осень 2021',
 'пальто осень серое',
 'пальто осень весна',
 'пальто осеннее драп',
 'пальто осеннее 2021',
 'пальто осеннее твоё',
 'пальто осень теплое',
 'пальто осень жёлтое',
 'пальто осень шерсть',
 'пальто осень прямое',
 'пальто осень балонь',
 'пальто осеннее яркое',
 'пальто осеннее синар',
 'пальто осеннеешерсть',
 'пальто осенне зимнее',
 'пальто осень женское',
 'пальто осень мужское',
 'пальто осень оверсайз',
 'пальто осенее дапово',
 'пальто осеннее черное',
 'пальто осеннее теплое',
 'пальто осеннее зарина',
 'пальто осенний бордовый',
 'пальто осень стёганое',
 'пальто осень синтепон',
 'пальто осеннее женское',
 'пальто осеннее длинные',
 'пальто осеннее длинное',
 'пальто осеннее меховое',
 'пальто осеннее мужское',
 'пальто осеннее вязаное',
 'пальто осеннее овер

#  Имплементация алгоритма ранжирования

Применяется ранжирование по популярности (два подхода):
1. Расчет эмбедингов для всех популярных запросов
2. Для нового запроса находим ближайшие из популярных
3. Расчитываем коэффициент ранжирования, исходя из похожести и рейтинга популярности (<h style="color:yellow;">TODO</h>)

In [ ]:
# ! wget https://raw.githubusercontent.com/diogodanielsoaresferreira/document_representations_tests/master/models/models.py

In [18]:
! pip install -qqq sentence_transformers

     |████████████████████████████████| 78 kB 3.0 MB/s 
     |████████████████████████████████| 3.1 MB 12.0 MB/s 
     |████████████████████████████████| 3.3 MB 57.6 MB/s 
     |████████████████████████████████| 1.2 MB 41.3 MB/s 
     |████████████████████████████████| 59 kB 3.8 MB/s 
     |████████████████████████████████| 596 kB 53.8 MB/s 
     |████████████████████████████████| 895 kB 61.8 MB/s 


In [19]:
import nltk
import torch
import numpy as np
import pandas as pd
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
# from models import InferSent
# from gensim.models.doc2vec import Doc2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
nltk.download('punkt')

# Number of top news
K=5

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [20]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [21]:
item_description = popularity["query"].astype(str)
ratings = popularity["query_popularity"].astype(int)
print(item_description)
assert len(item_description) == len(ratings)

0                            ноутбук
1             куртка женская осенняя
2                    ботинки женские
3                         видеокарта
4             пальто женское осеннее
                     ...            
336983    конструктор робо-динозавры
336984                          чика
336985                 Дешевый товар
336986                         макса
336987             чехол для телефон
Name: query, Length: 336988, dtype: object


## TF-IDF
В первом подходе для создания эмбедингов мы используем TF-IDF. Для запросов мы делаем препроцессинг, переводя их в нижний регистр и лемматизируем с помощью WordNet, затем создаем матрицу TF-IDF.

In [22]:
lemmatizer = WordNetLemmatizer()
tfidf_vectorizer = TfidfVectorizer()

In [23]:
descriptions_lemmatized = [" ".join([lemmatizer.lemmatize(token.lower()) for token in word_tokenize(description)]) for description in item_description.values ]

In [24]:
descriptions_representation_tfidf = tfidf_vectorizer.fit_transform(descriptions_lemmatized)

## Sentence-Bert
В этом подходе используются трансформеры Sentence Transformers. Мы получим эмбединги для текстов запросов с помощью предобученной мультиязыковой модели **distiluse-base-multilingual-cased-v1**.

In [25]:
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/556 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/452 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/114 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

In [ ]:
# embeddings_distilbert = model.encode(item_description.values)

In [ ]:
# import pickle as pk

# file_path = "embeddings.pk"
# with open(file_path, 'ab+') as fp:
#     pk.dump(embeddings_distilbert, fp)

In [26]:
!gdown --id 1s29EgnTG_jNRrL-KXE-hq7VGOC7w-nwj

Downloading...
From: https://drive.google.com/uc?id=1s29EgnTG_jNRrL-KXE-hq7VGOC7w-nwj
To: /content/embeddings.pk
100% 690M/690M [00:05<00:00, 127MB/s]


In [27]:
import pickle as pk

file_path = "embeddings.pk"
with open(file_path, 'rb') as fp:
    embeddings_distilbert = pk.load(fp)

## Сравнение результатов

In [28]:
def find_similar(vector_representation, all_representations, k=1):
    similarity_matrix = cosine_similarity(vector_representation, all_representations)
    np.fill_diagonal(similarity_matrix, 0)
    similarities = similarity_matrix[0]
    if k == 1:
        print(min(similarities))
        return [np.argmax(similarities)]
    elif k is not None:
        print(np.sort(similarities)[-k:])
        return np.flip(similarities.argsort()[-k:][::1])
    

Сравним результаты, полученные с помощью первого и второго подхода.

In [29]:
descriptions = ["ботинки женские мембрана", 
                "обувь женская ОКСФОРДЫ", 
                "противоударный чехол на айфон 6",
                "зимняя обувь для мальчиков 37-38 размеров",
                "бисер",
                "робот змея",
                "костюм на флисе женский",
                "светящийся ошейник для собак",
                "изделия из мед сплава",
                "ремкомплект раздатки Нива Chevrolet"]

In [30]:
for description in descriptions:
    printmd(f"## ТЕКСТ ЗАПРОСА: {description}")
    print()
    
    tf_idf_similar_indexes = find_similar(tfidf_vectorizer.transform([" ".join([lemmatizer.lemmatize(token.lower()) for token in word_tokenize(description)])]), descriptions_representation_tfidf, K)
    printmd("* 5 most similar descriptions using TF-IDF")
    for index in tf_idf_similar_indexes:
        printmd(f"{item_description[index]} - **рейтинг {ratings[index]}**")
    print()
    
    distilbert_similar_indexes = find_similar(model.encode([description]), embeddings_distilbert, K)
    printmd("* 5 most similar descriptions using Sentence-Bert")
    for index in distilbert_similar_indexes:
        printmd(f"{item_description[index]} - **рейтинг {ratings[index]}**")
    print()

## ТЕКСТ ЗАПРОСА: ботинки женские мембрана


[0.77100126 0.80751622 0.81347734 0.81347734 0.92894633]


* 5 most similar descriptions using TF-IDF

ботинки мембрана - **рейтинг 10**

ботинки мембрана для мальчика - **рейтинг 10**

...

Водительский коврик автомобильный для Chevrolet Lacetti - **рейтинг 7**

дефлектор капота chevrolet niva - **рейтинг 7**

Резиновый коврик на Chevrolet Cruze - **рейтинг 6**

Чехол га ремне айфон - **рейтинг 8**

EVA коврики на Chevrolet Lacetti 2004 - 2013 - **рейтинг 8**

# Пример использования для автозаполнения

In [31]:
def calculate_popularity_rank(vector_representation, all_representations, all_ratings, k=1):
    similarity_matrix = cosine_similarity(vector_representation, all_representations)
    np.fill_diagonal(similarity_matrix, 0)
    similarities = similarity_matrix[0]
    if k > 0:
        sim = np.sort(similarities)[-k:]
        ids = np.flip(similarities.argsort()[-k:][::1])
        ranks = all_ratings[ids]
        # print(f"sim:{sim}, rank:{ranks}")
        return sim * ranks

In [32]:
def sort_descriptions(descriptions, model="tfidf"):
  ranks = []
  for description in descriptions:
      # printmd(f"ТЕКСТ ЗАПРОСА: {description}")
      if model == "tfidf":
          word_embedding = tfidf_vectorizer.transform([" ".join([lemmatizer.lemmatize(token.lower()) for token in word_tokenize(description)])])
          tf_idf_similar_indexes = calculate_popularity_rank(word_embedding, descriptions_representation_tfidf, ratings, 1)
          ranks.append(tf_idf_similar_indexes.tolist()[0])
      
      else:
          word_embedding = model.encode(sentences=[description]) 
          distilbert_similar_indexes = calculate_popularity_rank(word_embedding, embeddings_distilbert, ratings,1)
          ranks.append(distilbert_similar_indexes.tolist()[0])
  sorted_desriptions = [x for _, x in sorted(zip(ranks, descriptions), reverse=True)]

  return sorted_desriptions, ranks

In [33]:
ranks = sort_descriptions(descriptions)
print(ranks[0], "\n", ranks[1])

ranks = sort_descriptions(descriptions, model=model)
print(ranks[0], "\n", ranks[1])

['светящийся ошейник для собак', 'бисер', 'ботинки женские мембрана', 'костюм на флисе женский', 'обувь женская ОКСФОРДЫ', 'зимняя обувь для мальчиков 37-38 размеров', 'робот змея', 'ремкомплект раздатки Нива Chevrolet', 'противоударный чехол на айфон 6', 'изделия из мед сплава'] 
 [9.289463276504648, 7.81187395217782, 2.0, 6.208218074898757, 10.0, 6.0, 9.0, 10.0, 1.502052600163039, 2.2942830275611468]
['светящийся ошейник для собак', 'бисер', 'костюм на флисе женский', 'противоударный чехол на айфон 6', 'зимняя обувь для мальчиков 37-38 размеров', 'ботинки женские мембрана', 'обувь женская ОКСФОРДЫ', 'робот змея', 'ремкомплект раздатки Нива Chevrolet', 'изделия из мед сплава'] 
 [8.390976011753082, 8.154578447341919, 9.036405086517334, 8.701983511447906, 9.999999403953552, 5.999998927116394, 9.999998807907104, 10.0, 0.7927213311195374, 4.472672462463379]


In [63]:
def is_latin(word):
    abc = "abcdefghijklmnopqrstuvwxyz"
    for letter in word:
        if letter in abc:
            return True
    return False

def postprocess_descriptions(descriptions, delete_brand=True, delete_letters=True, shorten=True):
    for i in range(0, len(descriptions)):
        if delete_brand:
            descriptions[i] = ' '.join([w for w in descriptions[i].split() if not is_latin(w)])
        if delete_letters:
            descriptions[i] = ' '.join([w for w in descriptions[i].split() if len(w)>1])

In [64]:
searches = get_searches("пальто осе")
searches = sort_descriptions(searches)[0]

In [65]:
postprocess_descriptions(searches, delete_brand=False)
searches[:20]

['пальто осень-зима',
 'пальто осеннее теплое',
 'пальто осеннее капюшоном',
 'пальто осеннее оверсайз',
 'пальто осеннее мужское',
 'пальто осень-весна женское',
 'пальто осень синтепон',
 'пальто осень',
 'пальто осень женское длинное',
 'пальто осень женское',
 'пальто осень балонь',
 'пальто осенние женское',
 'пальто осеннеешерсть',
 'пальто осеннее стеганое женское',
 'пальто осеннее женское',
 'пальто осенне женское',
 'пальто осенее дапово',
 'пальто осеннеедля девочки',
 'пальто осеннее на',
 'adona sandrelli пальто осень']